In [1]:
pip install networkx


Note: you may need to restart the kernel to use updated packages.


In [5]:
import networkx as nx
G = nx.Graph()
G.add_edge('A', 'B', base_time=10)
G.add_edge('B', 'C', base_time=15)
G.add_edge('A', 'C', base_time=30)
G.add_edge('C', 'D', base_time=20)
G.add_edge('B', 'D', base_time=25)
traffic_volumes = {
    ('A', 'B'): 3000,
    ('B', 'C'): 5000,
    ('A', 'C'): 1000,
    ('C', 'D'): 4500,
    ('B', 'D'): 3500,
}

max_volume = 6000  
alpha = 0.5        
for u, v, data in G.edges(data=True):
    volume = traffic_data.get((u, v)) or traffic_data.get((v, u)) or 0
    congestion_factor = 1 + alpha * (volume / max_volume)
    data['adjusted_time'] = data['base_time'] * congestion_factor
    start = 'A'
end = 'D'
path = nx.shortest_path(G, source=start, target=end, weight='adjusted_time')
time = nx.shortest_path_length(G, source=start, target=end, weight='adjusted_time')

print(f"Suggested route: {path}")
print(f"Estimated travel time: {time:.2f} minutes")




Suggested route: ['A', 'B', 'D']
Estimated travel time: 44.79 minutes


In [6]:
def suggest_route(graph, traffic_data, start, end, max_volume=6000, alpha=0.5):
    for u, v, data in graph.edges(data=True):
        volume = traffic_data.get((u, v)) or traffic_data.get((v, u)) or 0
        congestion_factor = 1 + alpha * (volume / max_volume)
        data['adjusted_time'] = data['base_time'] * congestion_factor
    path = nx.shortest_path(graph, source=start, target=end, weight='adjusted_time')
    time = nx.shortest_path_length(graph, source=start, target=end, weight='adjusted_time')
    
    return path, time
route, travel_time = suggest_route(G, traffic_volumes, 'A', 'D')
print(f"Route: {route}, Estimated time: {travel_time:.2f} minutes")


Route: ['A', 'B', 'D'], Estimated time: 44.79 minutes


In [7]:
predicted_traffic = {
    ('A', 'B'): 3200,
    ('B', 'C'): 4800,
    ('A', 'C'): 1100,
    ('C', 'D'): 4300,
    ('B', 'D'): 3700,
}
route, travel_time = suggest_route(G, predicted_traffic, 'A', 'D')
print(f"Updated Route: {route}, Estimated time: {travel_time:.2f} minutes")


Updated Route: ['A', 'B', 'D'], Estimated time: 45.38 minutes


In [11]:
def get_valid_node(prompt, valid_nodes):
    while True:
        node = input(prompt).strip()
        if node in valid_nodes:
            return node
        else:
            print(f"Invalid location! Please enter one of: {', '.join(valid_nodes)}")

valid_nodes = list(G.nodes)

start = get_valid_node("Enter start location: ", valid_nodes)
end = get_valid_node("Enter destination: ", valid_nodes)

route, travel_time = suggest_route(G, predicted_traffic, start, end)
print(f"Route: {route}, Estimated time: {travel_time:.2f} minutes")


Enter start location:  A
Enter destination:  D


Route: ['A', 'B', 'D'], Estimated time: 45.38 minutes


In [12]:
try:
    route, travel_time = suggest_route(G, predicted_traffic, start, end)
    print(f"Route: {route}, Estimated time: {travel_time:.2f} minutes")
except nx.NetworkXNoPath:
    print("No available path between the given locations.")
except Exception as e:
    print(f"An error occurred: {e}")


Route: ['A', 'B', 'D'], Estimated time: 45.38 minutes


In [13]:
import networkx as nx

def suggest_alternative_routes(graph, traffic_data, start, end, max_volume=6000, alpha=0.5, max_alternatives=3):
    # Update adjusted travel times based on traffic volume
    for u, v, data in graph.edges(data=True):
        volume = traffic_data.get((u, v)) or traffic_data.get((v, u)) or 0
        congestion_factor = 1 + alpha * (volume / max_volume)
        data['adjusted_time'] = data['base_time'] * congestion_factor

    # Generate k-shortest paths (using Yen's algorithm or simple approach)
    try:
        from networkx.algorithms.simple_paths import shortest_simple_paths
    except ImportError:
        print("NetworkX version may not support shortest_simple_paths")
        return []

    # Get generator of simple paths sorted by weight (adjusted_time)
    paths_generator = shortest_simple_paths(graph, source=start, target=end, weight='adjusted_time')

    alternatives = []
    for path in paths_generator:
        time = 0
        # Calculate total travel time for the path
        for i in range(len(path) - 1):
            edge_data = graph.get_edge_data(path[i], path[i+1])
            time += edge_data['adjusted_time']
        alternatives.append((path, time))
        if len(alternatives) >= max_alternatives:
            break

    return alternatives

# Example usage
alternatives = suggest_alternative_routes(G, predicted_traffic, 'A', 'D')
for idx, (route, time) in enumerate(alternatives, 1):
    print(f"Option {idx}: Route: {route}, Estimated Time: {time:.2f} minutes")


Option 1: Route: ['A', 'B', 'D'], Estimated Time: 45.38 minutes
Option 2: Route: ['A', 'C', 'D'], Estimated Time: 59.92 minutes
Option 3: Route: ['A', 'B', 'C', 'D'], Estimated Time: 60.83 minutes


In [14]:
def choose_route_option(alternatives):
    print("\nAvailable route options:")
    for idx, (route, time) in enumerate(alternatives, 1):
        print(f"{idx}. Route: {route}, Estimated Time: {time:.2f} minutes")

    while True:
        choice = input(f"Select route option (1-{len(alternatives)}): ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(alternatives):
            return alternatives[int(choice) - 1]
        else:
            print("Invalid choice. Please try again.")

# Assuming alternatives variable from previous step
selected_route, selected_time = choose_route_option(alternatives)
print(f"\nYou selected route: {selected_route} with estimated travel time {selected_time:.2f} minutes")



Available route options:
1. Route: ['A', 'B', 'D'], Estimated Time: 45.38 minutes
2. Route: ['A', 'C', 'D'], Estimated Time: 59.92 minutes
3. Route: ['A', 'B', 'C', 'D'], Estimated Time: 60.83 minutes


Select route option (1-3):  3



You selected route: ['A', 'B', 'C', 'D'] with estimated travel time 60.83 minutes
